In [4]:
def preprocessing(prediction):

    for i in prediction:
            pre_ans = i.argmax()  # 예측 레이블
            print(i)
            print(pre_ans)
            pre_ans_str = ''
            if pre_ans == 0: pre_ans_str = "부라노섬"
            elif pre_ans == 1: pre_ans_str = "에펠탑"
            elif pre_ans == 2: pre_ans_str = "홀슈타트호수"
            elif pre_ans == 3: pre_ans_str = "홀슈타트광장"    
            elif pre_ans == 4: pre_ans_str = "레이크루이스"
            elif pre_ans == 5: pre_ans_str = "런던아이"
            elif pre_ans == 6: pre_ans_str = "동방명주"
            elif pre_ans == 7: pre_ans_str = "포지타노"   
            elif pre_ans == 8: pre_ans_str = "산토리니"  
                
            else: pre_ans_str = "잔세스칸스"

            if i[0] >= 0.7 : 
                predict_name=pre_ans_str
            if i[1] >= 0.7:
                predict_name=pre_ans_str
            if i[2] >= 0.7: 
                predict_name=pre_ans_str
            if i[3] >= 0.7:
                predict_name=pre_ans_str
            if i[4] >= 0.7 :
                predict_name=pre_ans_str
            if i[5] >= 0.7: 
                predict_name=pre_ans_str
            if i[6] >= 0.7: 
                predict_name=pre_ans_str
            if i[7] >= 0.7: 
                predict_name=pre_ans_str
            if i[8] >= 0.7: 
                predict_name=pre_ans_str
            if i[9] >= 0.7: 
                predict_name=pre_ans_str
    return predict_name

In [5]:
def ko_preprocessing(ko_prediction):
    ko_predict_name =[]
    pre_ans_str = []
    for i in ko_prediction:
        if i[0] >= 0.3 : 
            ko_predict_name.append("가평_쁘띠프랑스")
        if i[1] >= 0.3:
            ko_predict_name.append("가평_에델바이스")
        if i[2] >= 0.4: 
            ko_predict_name.append("거제_바람의언덕")
        if i[3] >= 0.3:
            ko_predict_name.append("남해_다랭이마을")
        if i[4] >= 0.12:
            ko_predict_name.append("남해_독일마을")
        if i[5] >= 0.15: 
            ko_predict_name.append("부산_감천마을")
        if i[6] >= 0.2: 
            ko_predict_name.append("부산_더베이")
        if i[7] >= 0.02: 
            ko_predict_name.append("부산_장림포구")
        if i[8] >= 0.2: 
            ko_predict_name.append("삼척_장호항") 
        if i[9] >= 0.8: 
            ko_predict_name.append("서울_남산타워")
        if i[10] >= 0.3:
            ko_predict_name.append("송도_센트럴파크")
        if i[11] >= 0.3: 
            ko_predict_name.append("아산_지중해마을")
        if i[12] >= 0.6:
            ko_predict_name.append("인천_월미도")
        if i[13] >= 0.0015 :
            ko_predict_name.append("제부도_산토리니펜션")
        if i[14] >= 0.4: 
            ko_predict_name.append("제주_용머리") 
        if i[15] >= 0.009: 
            ko_predict_name.append("춘천_산토리니카페")
        if i[16] >= 0.4: 
            ko_predict_name.append("화순_세량지")

    return ko_predict_name

In [6]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request,redirect
import pickle #데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np
!pip install cx_Oracle --upgrade
import cx_Oracle
import time
from PIL import Image
import os, glob, numpy as np
from tensorflow.keras.models import load_model
import os, os.path, shutil
import zipfile
from tensorflow.keras import optimizers

app = Flask(__name__)#내장변수 name을 이용해 서버를 구동시키는 객체 생성
@app.route("/predict/predict", methods=['GET'])
def predict_survived() :
    if request.method == 'GET' : 
        print(request.args)
        dsn = cx_Oracle.makedsn("localhost", 1521, "hr")
        db = cx_Oracle.connect("hr", "hr", dsn)
        cursor = db.cursor()
        old_raw = 0
        
        caltech_dir='C:/Users/smhrd/Desktop/algoshipda/.metadata/.plugins/org.eclipse.wst.server.core/tmp0/wtpwebapps/Algo/upload/'
        image_w = 150
        image_h = 150

        pixels = image_h * image_w * 3

        X = []
        filename = request.args['fileName']
        
        print(filename)
        print(filename.encode('euc-kr').decode('euc-kr'))
        print(filename.encode('utf-8').decode('utf-8'))
        
        file = caltech_dir+filename
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)

        X = np.array(X)
        
        model=load_model('overseas_trip_model.h5')
        ko_model=load_model('korea_trip_model5_2.h5')
        prediction = model.predict(X)
        ko_prediction = ko_model.predict(X)
        ko_predict_name=[]
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        print(ko_prediction)
        predict_name = preprocessing(prediction)
        ko_predict_name = ko_preprocessing(ko_prediction)
        #이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
        
        print(predict_name)
        for i in ko_predict_name:
            print(i)
        execute = """update files set predictlocation=:predict_name where filename=:filename"""
        cursor.execute(execute, [predict_name,filename])
        db.commit()
        ko_place=''
        for i in range(len(ko_predict_name)):
            if i==(len(ko_predict_name)-1):
                url='KOREAPLACE='+ko_predict_name[i] 
                ko_place+=url
            else:
                url='KOREAPLACE='+ko_predict_name[i]+'&'
                ko_place+=url
        return redirect("http://192.168.43.59:8082/Algo/RecommendPage.jsp?OVERSEAPLACE="+predict_name+"&"+ko_place)
                       
#         return redirect("http://172.30.1.50:8082/Algo/RecommendPage.jsp?OVERSEAPLACE="+predict_name)
if __name__ == "__main__" :   # .py 파일에서 main 함수 역할하는 코드
    app.run(host="192.168.43.59",port="9000")

Requirement already up-to-date: cx_Oracle in c:\users\smhrd\anaconda3\envs\deep_gpu\lib\site-packages (8.0.1)
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.43.59:9000/ (Press CTRL+C to quit)


ImmutableMultiDict([('fileName', 'pearl1.jpg')])
pearl1.jpg
pearl1.jpg
pearl1.jpg


192.168.43.59 - - [26/Oct/2020 12:19:31] "GET /predict/predict?fileName=pearl1.jpg HTTP/1.1" 302 -


[[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000]]
[0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
6
동방명주
서울_남산타워
ImmutableMultiDict([('fileName', 'pearl2.jpg')])
pearl2.jpg
pearl2.jpg
pearl2.jpg


...

[[0.005 0.000 0.000 0.034 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.005
  0.000 0.000 0.078 0.008 0.870]]
[0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
4
레이크루이스
화순_세량지
ImmutableMultiDict([('fileName', 'lake32.jpg')])
lake32.jpg
lake32.jpg
lake32.jpg


192.168.43.59 - - [26/Oct/2020 12:23:29] "GET /predict/predict?fileName=lake32.jpg HTTP/1.1" 302 -


[[0.005 0.000 0.000 0.034 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.005
  0.000 0.000 0.078 0.008 0.870]]
[0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
4
레이크루이스
화순_세량지
ImmutableMultiDict([('fileName', 'santorini19.jfif')])
santorini19.jfif
santorini19.jfif
santorini19.jfif


192.168.43.59 - - [26/Oct/2020 12:32:41] "GET /predict/predict?fileName=santorini19.jfif HTTP/1.1" 302 -


[[0.001 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.006 0.000 0.000 0.960
  0.000 0.002 0.000 0.031 0.000]]
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
8
산토리니
아산_지중해마을
제부도_산토리니펜션
춘천_산토리니카페
